<a href="https://colab.research.google.com/github/MeganMulholland/SpellChecker/blob/main/SpellChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spell Checker
Software Engineering Final Project

Megan Mulholland, Jake Hauff, Ayo Abraham, Connor Flok


similar project to look at: https://github.com/arthurflor23/spelling-correction

In [9]:
!pip install textblob
from textblob import TextBlob

import nltk

# Download required NLTK corpora
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
from textblob import TextBlob

# Example text with spelling errors
text = "This is an exampel of a sentance with sum mispelled words."

# Create a TextBlob object
blob = TextBlob(text)

# Correct the spelling
corrected_text = blob.correct()

print("Original Text:", text)
print("Corrected Text:", corrected_text)

Original Text: This is an exampel of a sentance with sum mispelled words.
Corrected Text: His is an example of a sentence with sum dispelled words.


Beginning real project:

In [ ]:
Main functions.

add_custom_word(word): Adds a word to the custom word dictionary.
spell_check(text): The main spell checker function. Returns the
corrected text, a list of corrections, and a dictionary of suggestions.

In [13]:
# custom dictionary
custom_words = set()

def add_to_custom_dictionary(word):
    custom_words.add(word.lower())

def spell_check(text):
    blob = TextBlob(text)

    corrections = []
    suggestions = {}
    corrected_words = []

    # process word-by-word to avoid TextBlob phrase bug
    for word in blob.words:
        word_lower = word.lower()

        # skip custom dictionary words
        if word_lower in custom_words:
            corrected_words.append(word)
            continue

        # Get suggestions from a *single-word* TextBlob
        word_blob = TextBlob(word)
        spellings = word_blob.spellcheck()   # NOW VALID

        best_guess, confidence = spellings[0]

        if best_guess.lower() != word_lower:
            corrections.append((word, best_guess))
            suggestions[word] = spellings

        corrected_words.append(best_guess)

    corrected_text = " ".join(corrected_words)
    return corrected_text, corrections, suggestions

UI

In [14]:
def run_checker():
    print("=== Simple Spell Checker ===\n")
    text = input("Enter text here:\n")

    corrected_text, corrections, suggestions = spell_check(text)

    print("\n--- Corrections ---")
    if not corrections:
        print("No spelling errors detected.")
    else:
        for wrong, correct in corrections:
            print(f"{wrong} → {correct}")
            print("  Suggestions:", suggestions[wrong])
            print()

    print("--- Corrected Text ---")
    print(corrected_text)


In [15]:
run_checker()

=== Simple Spell Checker ===

Enter text here:
Helo my fovorite thing to do is eat chococolate. Plese let me know WhAt you think


AttributeError: 'TextBlob' object has no attribute 'spellcheck'

Trying PySpell instead. More accurate and don't have to degrade textBlob version.

In [18]:
!pip install pyspellchecker
from spellchecker import SpellChecker


In [23]:

# Initialize spell checker
spell = SpellChecker()

# Allow user-defined words
custom_words = set()

def add_to_custom_dict(word):
    custom_words.add(word.lower())
    print(f"Added '{word}' to custom dictionary.")

def spell_check(text):
    words = text.split()
    corrected_words = []
    corrections = []
    suggestions = {}

    for word in words:
        lower = word.lower()

        # Skip custom words
        if lower in custom_words:
            corrected_words.append(word)
            continue

        # Skip punctuation-only tokens
        if all(not c.isalpha() for c in lower):
            corrected_words.append(word)
            continue

        # If the word is valid, keep as-is
        if lower in spell:
            corrected_words.append(word)
            continue

        # If misspelled:
        correct = spell.correction(lower)

        # If spellchecker returns None, fall back to original word
        if correct is None:
            correct = word

        candidate_suggestions = spell.candidates(lower)

        if correct != lower:
            corrections.append((word, correct))
            suggestions[word] = candidate_suggestions

        # Ensure we always append a string
        corrected_words.append(str(correct))

    corrected_text = " ".join(corrected_words)
    return corrected_text, corrections, suggestions



def run_spell_checker():
    print("Hello, welcome to the MPM Spell Checker!")
    text = input("Enter text here:\n")

    corrected_text, corrections, suggestions = spell_check(text)

    print("\n Corrections")
    if not corrections:
        print("No spelling errors found! Great job.")
    else:
        for wrong, correct in corrections:
            print(f"{wrong} → {correct}")
            print("  Suggestions:", suggestions[wrong])

    print("\n Corrected Text")
    print(corrected_text)

    print("\n Summary of Corrections")
    print(f"Total corrections made: {len(corrections)}")
    for wrong, correct in corrections:
        print(f"- {wrong} → {correct}")

In [25]:
run_spell_checker()

Hello, welcome to the MPM Spell Checker!
Enter text here:
Helloooooooo

 Corrections
Helloooooooo → Helloooooooo
  Suggestions: None

 Corrected Text
Helloooooooo

 Summary of Corrections
Total corrections made: 1
- Helloooooooo → Helloooooooo


Notes: added a part so that if it doesn't have a suggestion just still returns no suggestion. Can't yet handle very long letters at the end.